# This example uses the [Global News Dataset](https:\www.kaggle.com\datasets\everydaycodings\global-news-dataset) <span style="color: rgb(37, 51, 95); font-family: Merriweather, Georgia, serif; font-size: 16px;">from&nbsp;</span>  [Kaggle](https:\www.kaggle.com\) dataset.

In [ ]:
select top 10 n.source_name, n.title, n.content, n.VectorBinary_content from newsvector n

# **VECTOR\_DISTANCE FEATURE**

This function calculates the distance between two vectors using a specified distance metric.

The following distance metrics are supported:

- cosine – Cosine distance
    
- euclidean – Euclidean distance
    
- dot – (Negative) Dot product
    

Example Syntax: The following example creates a vector with three dimensions from a string with a JSON array

In [ ]:
DECLARE @v1 VARBINARY(8000) = JSON_ARRAY_TO_VECTOR('[1,1]')
DECLARE @v2 VARBINARY(8000) = JSON_ARRAY_TO_VECTOR('[-1,-1]')

SELECT 
    VECTOR_DISTANCE('euclidean', @v1, @v2) AS euclidean,
    VECTOR_DISTANCE('cosine', @v1, @v2) AS cosine,
    VECTOR_DISTANCE('dot', @v1, @v2) AS negative_dot_product;

In [ ]:
/*
    PROCEDURE: [dbo].[GET_EMBEDDINGS]
    DESCRIPTION:    This stored procedure is used to retrieve embeddings for a given text using the specified model. It makes use of an external REST endpoint to generate the embeddings.
    PARAMETERS:
    - @model: VARCHAR(MAX) - The name of the model to use for generating embeddings.    - @text: NVARCHAR(MAX) - The input text for which embeddings need to be generated.
    - @embedding: VARBINARY(8000) OUTPUT - The output parameter to store the generated embeddings.
    RETURNS:    None
    EXAMPLE USAGE:
    DECLARE @model VARCHAR(MAX) = '<model_name>';    DECLARE @text NVARCHAR(MAX) = '<input_text>';
    DECLARE @embedding VARBINARY(8000);    EXEC [dbo].[GET_EMBEDDINGS] @model, @text, @embedding OUTPUT;
*/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
--ALTER PROCEDURE [dbo].[GET_EMBEDDINGS]
(
    @model VARCHAR(MAX),
    @text NVARCHAR(MAX),
    @embedding VARBINARY(8000) OUTPUT
)
AS
BEGIN
    DECLARE @retval INT, @response NVARCHAR(MAX);
    DECLARE @url VARCHAR(MAX);
    DECLARE @payload NVARCHAR(MAX) = JSON_OBJECT('input': @text);

    -- Set the @url variable with proper concatenation before the EXEC statement
    SET @url = 'https://<Your App>.openai.azure.com/openai/deployments/' + @model + '/embeddings?api-version=2024-02-15-preview';

    EXEC dbo.sp_invoke_external_rest_endpoint 
        @url = @url,
        @method = 'POST',   
        @payload = @payload,   
        @headers = '{"Content-Type":"application/json", "api-key":"<Your Azure Open AI API Key"}', 
        @response = @response OUTPUT;

    -- Use JSON_QUERY to extract the embedding array directly
    DECLARE @jsonArray NVARCHAR(MAX) = JSON_QUERY(@response, '$.result.data[0].embedding');

    
    SET @embedding = JSON_ARRAY_TO_VECTOR(@jsonArray);
END

# How to embeddding a text to a vector?

In [ ]:
DECLARE @value_embedding NVARCHAR(MAX);

EXEC GET_VECTOR_EMBEDDINGS 'text-embedding-3-small', 'This text will be converted to a vector', @value_embedding OUTPUT;

Select @value_embedding

# How to embeddding a text to a binary?

In [ ]:
DECLARE @value_embedding VARBINARY(8000);

EXEC GET_EMBEDDINGS 'text-embedding-3-small', 'This text will be converted to a vector and a binary', @value_embedding OUTPUT;

Select @value_embedding

In [ ]:
-- This script converts the title and content of each article in the dbo.newsvector table 
-- to vector embeddings using the text-embedding-3-small model.
-- The generated embeddings are then stored in the VectorBinary_title and VectorBinary_content columns.
-- The script uses a cursor to iterate over each article that does not yet have embeddings.
alter table newsvector add embeddings VARBINARY(8000);

DECLARE @article_id INT;
DECLARE @content NVARCHAR(MAX);
DECLARE @content_embedding VARBINARY(8000);

-- Cursor to iterate over each article that does not yet have embeddings
DECLARE article_cursor CURSOR FOR
    SELECT article_id, title + ':' + full_content
    FROM dbo.newsvector
    WHERE embeddings IS NULL;

OPEN article_cursor;

FETCH NEXT FROM article_cursor INTO @article_id, @content;

WHILE @@FETCH_STATUS = 0
BEGIN    
    -- Generate the embedding for the article's content
    EXEC GET_EMBEDDINGS 'text-embedding-3-small', @content, @content_embedding OUTPUT;

    -- Update the VectorBinary_title and VectorBinary_content columns with the generated embeddings
    UPDATE dbo.newsvector
    SET embeddings = @content_embedding
    WHERE article_id = @article_id;

    -- Fetch the next article
    FETCH NEXT FROM article_cursor INTO @article_id, @content;
END;

CLOSE article_cursor;
DEALLOCATE article_cursor;
-- End of script


# Search on News Vector Content.

In [ ]:
	--Assuming you have a stored procedure to get embeddings for a given text
	DECLARE @e VARBINARY(8000);
	EXEC dbo.GET_EMBEDDINGS @model = 'text-embedding-3-small', @text = 'Generative AI', @embedding = @e OUTPUT;

	SELECT TOP(10) 
       VECTOR_DISTANCE('cosine', @e, VectorBinary_content) AS cosine_distance
      ,[source_name]
      ,[title]
      ,[full_content]
      ,[category]
      ,[published]
	FROM newsvector
	ORDER BY cosine_distance;